In [ ]:
import pdbufr
import pandas as pd
import numpy as np 
from openpyxl import load_workbook
from openpyxl.styles import Border, Side
from openpyxl.styles import Alignment

bufr_06 = "/home/lagha/data/bufr/synop_alg_202403250600.bufr"
bufr_18 = "/home/lagha/data/bufr/synop_alg_202403241800.bufr"

df_06 = pdbufr.read_bufr(bufr_06, columns=("stationOrSiteName", 
                                          "stationNumber",
                                          "heightOfStationGroundAboveMeanSeaLevel",
                                          "windDirection",
                                          "windSpeed",
                                          "cloudCoverTotal",
                                          "minimumTemperatureAtHeightAndOverPeriodSpecified"))

df_06_rr = pdbufr.read_bufr(bufr_06, columns=("stationOrSiteName", "totalPrecipitationOrTotalWaterEquivalent"),
                            filters={"timePeriod": -24})
df_06_ww = pdbufr.read_bufr(bufr_06, columns=("stationOrSiteName","presentWeather","pastWeather1","pastWeather2"))

df_18 = pdbufr.read_bufr(bufr_18, columns=("stationOrSiteName", "maximumTemperatureAtHeightAndOverPeriodSpecified"))

df = pd.merge(df_06, df_18[["stationOrSiteName", "maximumTemperatureAtHeightAndOverPeriodSpecified"]], on='stationOrSiteName', how='inner')
df = pd.merge(df, df_06_ww[["stationOrSiteName", "presentWeather"]], on='stationOrSiteName', how='inner')
df = pd.merge(df, df_06_ww[["stationOrSiteName", "pastWeather1"]], on='stationOrSiteName', how='inner')
df = pd.merge(df, df_06_ww[["stationOrSiteName", "pastWeather2"]], on='stationOrSiteName', how='inner')
df = pd.merge(df, df_06_rr[["stationOrSiteName", "totalPrecipitationOrTotalWaterEquivalent"]], on='stationOrSiteName', how='inner')

df.rename(columns={"stationOrSiteName":"STATIONS",                                     
                   "heightOfStationGroundAboveMeanSeaLevel":"ALTITUDE EN METRES",
                   "windDirection":"Dir","windSpeed":"Vit (m/s)",
                   "cloudCoverTotal":"Néb (1/8)",
                   "presentWeather":"ww",
                   "totalPrecipitationOrTotalWaterEquivalent":"Précip (mm)",
                   "maximumTemperatureAtHeightAndOverPeriodSpecified":"Max de la veille",
                   "minimumTemperatureAtHeightAndOverPeriodSpecified":"Min de la nuit",
                   "pastWeather1":"W1","pastWeather2":"W2"}, inplace=True)


# Fixed list of stations
stations = ["ADRAR", "AIN-BESSAM", "AIN-DEFLA", "AIN-SEFRA", "AIN-TEMOUCHENT", 
            "ALGER-PORT", "ANNABA", "ARZEW", "BARIKA", "BATNA", "B-B-ARRERIDJ", 
            "B-B-MOKHTAR", "BECHAR", "BEJAIA-AEROPORT", "BENI-ABBES", "BENI-SAF", 
            "BISKRA", "BOUCHEGOUF", "BOUHAROUN", "BOUIRA", "BOU-SAADA", "CHLEF", 
            "CONSTANTINE", "DAR-EL-BEIDA", "DELLYS", "DJANET", "DJELFA", "EL-BAYADH", 
            "EL-GOLEA", "EL-KHEITER", "EL-OUED", "EL-TARF", "GHARDAIA", "GHAZAOUET", 
            "GUELMA", "HASSI-MESSAOUD", "HASSI-RMEL", "ILLIZI", "IN-AMENAS", "IN-GUEZZAM", 
            "IN-SALAH", "JIJEL", "KHENCHELLA", "KSAR-CHELLALA", "LAGHOUAT", "MAGHNIA", 
            "MASCARA-GHRISS", "MECHERIA", "MEDEA", "MILA", "MILIANA", "MOSTAGANEM", 
            "MSILA", "NAAMA", "O-EL-BOUAGHI", "ORAN-PORT", "ORAN-SENIA", "ORAN-TAFARAOUI", 
            "OUARGLA", "RHOURD NOUSS", "SAIDA", "SETIF", "SIDI-BEL-ABBES", "SKIKDA", 
            "SOUK AHRAS", "TAMANRASSET", "TEBESSA", "TENES", "TIARET", "TIMIMOUN", 
            "TINDOUF", "TISSEMSILT", "TIZI-OUZOU", "TLEMCEN-ZENATA", "TOUGGOURT"]

df = df.dropna(subset = ["STATIONS"])
df = df.dropna(subset = ["ALTITUDE EN METRES"])

for station in stations:
    # Check if the station exists in the DataFrame
    if station not in df["STATIONS"].unique():
        # Create a DataFrame with a single row for the new station
        new_row = pd.DataFrame([{"STATIONS": station, 
                                  "ALTITUDE EN METRES": None, 
                                  "Dir": None, 
                                  "Vit (m/s)": None, 
                                  "Néb (1/8)": None, 
                                  "ww": None, 
                                  "Précip (mm)": None, 
                                  "W1": None, 
                                  "W2": None, 
                                  "Max de la veille": None, 
                                  "Min de la nuit": None}])
        # Concatenate the new DataFrame with the existing DataFrame
        df = pd.concat([df, new_row], ignore_index=True)

df = df.sort_values(["STATIONS"])

df["Néb (1/8)"] = (df["Néb (1/8)"] * 8 / 100).round()
df["Max de la veille"] = (df["Max de la veille"] - 273.15).round(1)
df["Min de la nuit"] = (df["Min de la nuit"] - 273.15).round(1)

df["ww"] = df["ww"].where(df["ww"] < 100, None)
df["W1"] = df["W1"].where(df["W1"] < 10, None)
df["W2"] = df["W2"].where(df["W2"] < 10, None)

def windRose(num):
    rose = np.array(["N", "NE", "E", "SE", "S", "SW", "W", "NW"])
    num = pd.to_numeric(num, errors='coerce')
    val = np.round(num / 45) % 8
    return [rose[int(v)] if not np.isnan(v) else np.nan for v in val]
    
dd = np.array(df["Dir"])  
ff = np.array(df["Vit (m/s)"])  

df["Dir"] = windRose(dd)

df.loc[df["Vit (m/s)"] == 0, "Dir"] = 'Calme'
#df_06.loc[(df_06["Vit (m/s)"] < 2) & (df_06["Vit (m/s)"] > 0), "Dir"] = 'VRB'

df = df[["STATIONS","ALTITUDE EN METRES","Dir","Vit (m/s)","Néb (1/8)","ww","Précip (mm)","W1","W2","Max de la veille","Min de la nuit"]]
#df = df[["STATIONS","Précip (mm)","Dir","Max de la veille","Min de la nuit"]]

# pd.set_option('display.max_rows', 500)
from IPython.display import display
with pd.option_context('display.max_rows', 100, 'display.max_columns', None):
    display(df) #need display to show the dataframe when using with in jupyter

exit

# Load Excel template
wb = load_workbook('template.xlsx')
ws = wb.active

# Insert DataFrame into Excel template starting from 4th row
start_row = 4
for row_index, row in enumerate(df.itertuples(index=False), start=start_row):
    for col_index, value in enumerate(row, start=1):
        cell = ws.cell(row=row_index, column=col_index)
        cell.value = value
        if col_index != 1:
            cell.alignment = Alignment(horizontal='center', vertical='center')
    
# Define border style
border_style = Border(left=Side(style='thin'),
                      right=Side(style='thin'),
                      top=Side(style='thin'),
                      bottom=Side(style='thin'))

# Apply border to all cells
for row in ws.iter_rows():
    
    for cell in row:
        cell.border = border_style

# Save to Excel
wb.save('output.xlsx')

In [ ]:
from windrose import WindroseAxes

# dd = np.array([d for d in dd if not np.isnan(d)])
# ff = np.array([f for f in ff if not np.isnan(f)])
from windrose import WindroseAxes
from matplotlib import pyplot as plt
import matplotlib.cm as cm
import numpy as np

# Create wind speed and direction variables

ff = np.random.random(500) * 6
dd = np.random.random(500) * 360


ax = WindroseAxes.from_ax()
ax.bar(dd, ff, normed=True, opening=1, edgecolor="white")
ax.set_legend()

In [ ]:
from windrose import WindroseAxes
from matplotlib import pyplot as plt
import matplotlib.cm as cm
import numpy as np

# Create wind speed and direction variables

ws = np.random.random(500) * 6
wd = np.random.random(500) * 360

ax = WindroseAxes.from_ax()
ax.bar(wd, ws, normed=True, opening=0.8, edgecolor='white')
ax.set_legend()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Step 1: Define wind directions and corresponding colors
directions = ['N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW']
colors = ['blue', 'cyan', 'green', 'yellow', 'orange', 'red', 'purple', 'magenta']

# Step 2: Define wind speed (example speed)
wind_speed = 10

# Step 3: Define degree ranges for each wind direction
degree_ranges = [(338, 23), (23, 68), (68, 113), (113, 158), 
                 (158, 203), (203, 248), (248, 293), (293, 338)]

# Step 4: Plot the wind rose using Matplotlib with colored sectors
fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})

theta = np.linspace(0, 2 * np.pi, len(directions), endpoint=False).tolist()
width = np.deg2rad(360 / len(directions))

bars = ax.bar(theta, [wind_speed] * len(directions), width=width, bottom=0.0, color=colors)

ax.set_theta_direction(-1)  # Set clockwise direction
ax.set_theta_zero_location('N')  # Set zero direction to the North

ax.set_xticks(theta)
ax.set_xticklabels([])  # Remove existing labels

# Add annotations indicating degree ranges for each wind direction
for direction, angle, (start, end) in zip(directions, theta, degree_ranges):
    mid_angle = (angle + np.pi/8) % (2 * np.pi)  # Adjust position slightly away from the center
    # Place the end values outside the rose plot and in a different color
    ax.text(mid_angle, wind_speed + 1.5, f'{end}°', ha='center', va='center', color='blue')


# ax.set_title('Wind Rose')

# Hide radial labels
ax.set_yticklabels([])

plt.savefig('rose_plot.png', dpi=300)
plt.show()

In [ ]:
from datetime import datetime, timedelta

import xarray as xr

import metpy.calc as mpcalc
from metpy.cbook import get_test_data
from metpy.io import metar
from metpy.plots.declarative import (BarbPlot, ContourPlot, FilledContourPlot, MapPanel,
                                     PanelContainer, PlotObs)
from metpy.units import units

# Select the area string
area = 'alge'

# Get the extent and project for the selected area
extent = named_areas[area].bounds
proj = named_areas[area].projection

# Plot a simple figure for the selected area
plt.figure(1, figsize=(10, 10))
ax = plt.subplot(111, projection=proj)
ax.set_extent(extent, ccrs.PlateCarree())
ax.add_feature(cfeature.STATES.with_scale('10m'), edgecolor='grey', linewidth=0.75)
ax.add_feature(cfeature.COASTLINE.with_scale('10m'), linewidth=1.1)
ax.add_feature(cfeature.BORDERS.with_scale('10m'), edgecolor='black')
ax.set_title(f'area={area}          name={named_areas[area].description}'
             f'\nproj={proj.coordinate_operation.method_name}')
plt.show()

In [ ]:
# RBI 20191130

import os.path
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.cm as cm


from windrose import WindroseAxes

ax = WindroseAxes.from_ax()    
ax.bar(dd, ff, normed=True, opening=0.8, edgecolor='white',cmap=cm.hot)
ax.set_legend()
plt.suptitle('Rosze')
plt.switch_backend('TkAgg')

plt.show()

In [ ]:
import pdbufr
import pandas as pd
import numpy as np 
from openpyxl import load_workbook
from openpyxl.styles import Border, Side
from openpyxl.styles import Alignment

#YY, MM, DD, HH = 2024, 2, 29, 6
#yy, mm, dd, hh = 2024, 2, 28, 18

#bufr_18 = f'/home/lagha/data/bufr/synop_alg_{YY:04d}{MM:02d}{DD:02d}{HH:02d}00.bufr'
#bufr_06 = f'/home/lagha/data/bufr/synop_alg_{yy:04d}{mm:02d}{dd:02d}{hh:02d}00.bufr'

bufr = '/home/lagha/data/bufr/temp_alg_202403230000.bufr'

#df_06 = pdbufr.read_bufr(bufr_06, 
                        #columns=("stationOrSiteName", "stationNumber", "latitude", "longitude")
                        #filters={"latitude": slice(18, 37), "longitude": slice(-8, 12)})
                        #filters={"stationNumber": [843, 925]})

df = pdbufr.read_bufr(bufr, columns=("stationNumber","latitude","longitude","data_datetime","heightOfStationGroundAboveMeanSeaLevel", "airTemperature","dewpointTemperature","nonCoordinateGeopotentialHeight", "pressure", "windSpeed", "windDirection"),
                        #filters={"latitude": slice(18, 37), "longitude": slice(-8, 12)})
                        #filters={"stationNumber": [843, 925]})
                      filters={"pressure": [100000, 92500, 85000, 70000, 60000, 50000, 40000, 30000, 25000, 20000, 15000, 10000, 7000, 5000, 3000, 2000, 1000]})

pd.set_option('display.max_rows', 500)
df["pressure"] = df["pressure"] / 100
df["airTemperature"] = df["airTemperature"] - 273.15
df["dewpointTemperature"] = df["dewpointTemperature"] - 273.15
#df = df.sort_values(["pressure"])

pd.DataFrame(df)

In [ ]:
df = pdbufr.read_bufr("hirs.bufr",
    columns=("data_datetime", "latitude", "longitude", "brightnessTemperature"),
    filters={"count": 1,
            "radiometerIdentifier": 6,
            "tovsOrAtovsOrAvhrrInstrumentationChannelNumber": 5})
df